In [ ]:
import torch
import tqdm
import matplotlib.pyplot as plt
from utils import display_img, process
from latent_dataset import WPlusLatentsDataset
from SAE import SparseAutoencoder
import dnnlib
import legacy
from latent_dataset import save_latents
from torch.utils.data import DataLoader
from evaluate_saes import test_reconstructions

In [ ]:
TRAIN_PATH = '/Users/spra/Desktop/Personal Projects/CV_final_project/G_w_train_latents'
train_data = WPlusLatentsDataset(TRAIN_PATH)
VAL_PATH = '/Users/spra/Desktop/Personal Projects/CV_final_project/G_w_val_latents'
val_data = WPlusLatentsDataset(VAL_PATH)
TEST_PATH = '/Users/spra/Desktop/Personal Projects/CV_final_project/G_w_test_latents'
test_data = WPlusLatentsDataset(TEST_PATH)
train_loader = DataLoader(train_data, batch_size = 64, shuffle = True)
val_loader = DataLoader(train_data, batch_size = 64, shuffle = True)
test_loader = DataLoader(test_data, batch_size = 64, shuffle = True)

In [ ]:
SAE = torch.load('/Users/spra/Desktop/Personal Projects/CV_final_project/Image-Steering-SAEs/sae1024_10epochs.pt', weights_only = False)

In [ ]:
imgs, reconstructed_imgs = test_reconstructions(G, SAE, test_loader, h = 4, w = 4, display = True)

In [ ]:
num_images = 128
sae = SAE

In [ ]:
noise = torch.randn((num_images, G.z_dim), device = device)
c = None
with torch.no_grad():
    w = G.mapping(noise, c)
    # to take out the middle dimension of 14
    w = w[:, 0, :]
    reconstructions, sparse_representations = sae(w)
    w = einops.repeat(w, 'b d -> b l d', l = 14)
    images = G.synthesis(w)
    # images should be a tensor of shape (batch_size, 3, 256, 256)
    # sparse_representations should be a tensor of shape (batch_size, 1024)

In [ ]:
freqs = sparse_representations.mean(dim = 0)
maxes, freq_indices = torch.topk(freqs, k = 20)
print(f'Maxes: {maxes}')
print(f'Indices: {freq_indices}')

In [ ]:
def visualize_latent(sparse_representations, images, idx):
    # top_indices and bottom_indices are tensors of shape (k, latent_dim), which is usually (16, 1024)
    topk, top_indices = torch.topk(sparse_representations, k = 16, dim = 0, largest = True)
    bottomk, bottom_indices = torch.topk(sparse_representations, k = 16, dim = 0, largest = False)
    # were only taking the topk for the latent we choose using idx
    top_chosen_indices = top_indices[:, idx]
    top_activations = topk[:, idx]
    bottom_chosen_indices = bottom_indices[:, idx]
    bottom_activations = bottomk[:, idx]
    top_images = process(images[top_chosen_indices])
    bottom_images = process(images[bottom_chosen_indices])
    return top_images, bottom_images, top_activations, bottom_activations

In [ ]:
top, bottom, top_activations, bottom_activations = visualize_latent(sparse_representations, images, 2)